In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

**Carregamento dos dados**

In [3]:
dataprueba = pd.read_csv('datasetProyFinal.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasetProyFinal.csv'

In [ ]:
dataprueba.head(n=5)

In [ ]:
data = dataprueba['ORSC']

**Gráfico de produção de petróleo**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(data, "ok")
plt.grid()
plt.plot([len(data)-500, len(data)-500], [min(data), max(data)], "--", linewidth=2 )
plt.title('Volumes de Produção de Petróleo - Dados Reais')
plt.xlabel('Dia')
plt.ylabel('Volumen (m3/dia)')

**Normalização dos dados**

In [ ]:
scaler = MinMaxScaler()
data_scalet = scaler.fit_transform(np.array(data).reshape(-1,1))
data_scalet = [u[0] for u in data_scalet]

**Gráfico de produção de petróleo com dados normalizados**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(data_scalet, "ok")
plt.grid()
plt.plot([len(data_scalet)-500, len(data_scalet)-500], [min(data_scalet), max(data_scalet)], "--", linewidth=2 )
plt.title('Volumes de Produção de Petróleo - Dados Normalizados')
plt.xlabel('Dia')
plt.ylabel('Volumen (m3/dia)')

**Sequenciamento dos dados**

In [4]:
P = 100

X = []
for t in range(P-1, 3123 - 1):
    aux = [data_scalet[t-n] for n in range(P)]
    L = [aux[i] for i in range(len(aux)-1, -1, -1)]
    X.append(L)

observed_scalet= data_scalet[P:]

NameError: name 'data_scalet' is not defined

In [ ]:
datadf = pd.concat([pd.DataFrame(X), pd.DataFrame(observed_scalet, columns=['target'])], axis= 1)
datadf

**Rede neural**

In [ ]:
np.random.seed(123456)

In [ ]:
regressor = MLPRegressor(
        hidden_layer_sizes=(80,60,40,20),
        activation='relu',
        solver='lbfgs',
        alpha=0.1,
        batch_size=64,
        learning_rate='constant',
        learning_rate_init=0.01,
        max_iter=30000,
        shuffle=True,
        tol=0.0001,
        verbose=False,
        early_stopping=True,
        validation_fraction=0.3)

**Treinamento de rede neural**

In [ ]:
X_train = X[0:2623]
Y_train =  observed_scalet[0:2623]
regressor.fit(X_train, Y_train)

**Previçoes**

In [ ]:
pred = regressor.predict(X)

**Resultados e indicadores**

In [ ]:
test_set_rsquared = regressor.score(X, observed_scalet)
test_set_mae = mean_absolute_error(observed_scalet, pred)
test_set_mse = mean_squared_error(observed_scalet, pred)
test_set_rmse = np.sqrt(mean_squared_error(observed_scalet, pred))
expl_var_Y = explained_variance_score(observed_scalet, pred)

print('R_squared value: ', test_set_rsquared)
print('MAE: ', test_set_mae)
print('MSE: ', test_set_mse)
print('RMSE: ', test_set_rmse)
print('EVS: ', expl_var_Y)

**Explicação da variância pelo modelo**

In [ ]:
plt.figure(figsize=(6,4))
plt.scatter(observed_scalet,pred)
plt.plot(observed_scalet,observed_scalet,'r')

## Previções

**Gráfico do conjunto de dados com previçoes**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(data_scalet, "ok")
plt.grid()
plt.plot([None] * P + pred.tolist(),"or")
plt.plot([len(data_scalet)-500, len(data_scalet)-500], [min(data_scalet), max(data_scalet)], "--", linewidth=2 )
plt.title('Volumes de Produção de Petróleo - Dados Normalizados - Rede Neural-MLP')
plt.xlabel('Dia')
plt.ylabel('Volumen (m3/dia)')
plt.legend(['Valor real', 'Previção'])

**Transformação inversa de dados**

In [ ]:
ym = scaler.inverse_transform([[u] for u in pred])
ym =[u[0] for u in ym]

In [ ]:
observed = scaler.inverse_transform([[u] for u in observed_scalet])
observed =[u[0] for u in observed]

**Gráfico dos dados com suas previçoes (valores reais)**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(data, "ok")
plt.grid()
plt.plot([None] * P + ym,"or")
plt.plot([len(data)-500, len(data)-500], [min(data), max(data)], "--", linewidth=2 )
plt.title('Volumes de Produção de Petróleo - Dados Reais - Rede Neural-MLP')
plt.xlabel('Dia')
plt.ylabel('Volumen (m3/dia)')
plt.legend(['Valor real', 'Previção'])

In [ ]:
dataSeriedf = pd.concat([pd.DataFrame(observed, columns=['real']), pd.DataFrame(ym, columns=['pred'])], axis = 1)

In [ ]:
dataSeriedf

In [ ]:
dataSeriedf.to_csv('dataSerieRNdf.csv', index=False)

In [ ]:
dataSeriedf.iloc[2000]